In [36]:
# use docker to parse grib file
#docker pull osgeo/gdal
#docker run --rm -v "C:/Users/WeilunShi/OneDrive - CGIAR/Desktop/Food Crisis and Insecurity/0.external_data/ERA5_data:/data" osgeo/gdal:latest gdal_translate -of GTiff /data/ERA_2.grib /data/output_2.tif

import numpy as np
import rasterio
import geopandas as gpd
from rasterio.features import shapes
from shapely.geometry import shape

# Read the .tif file with Rasterio
with rasterio.open(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\0.external_data\ERA5_data\output_2.tif') as src:
    image = src.read(1)  # Read the first band
    transform = src.transform

# Convert the image data type to int16
image = image.astype(np.int16)
# Extract shapes from the raster data
shapes_and_values = list(shapes(image, transform=transform))
geometries = [shape(s[0]) for s in shapes_and_values]
values = [s[1] for s in shapes_and_values]

# Create a GeoDataFrame
gdf = gpd.GeoDataFrame({'geometry': geometries, 'value': values})

# Set the Coordinate Reference System (CRS) to match the input raster
gdf.crs = src.crs


# set CRS to 4236
gdf = gdf.to_crs(epsg=4326)



# read geojson
areas = gpd.read_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\output5.geojson')


# spatial join
result = gpd.sjoin(areas, gdf, how='left', op='intersects')


c:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\.venv\Lib\site-packages\IPython\core\interactiveshell.py:3445: FutureWarning: The `op` parameter is deprecated and will be removed in a future release. Please use the `predicate` parameter instead.
  if await self.run_code(code, result, async_=asy):


In [37]:
# rename
result = result.rename(columns={'value': 'temperature'})

#parse missing value
result.iloc[result['temperature']==9999] = np.nan

# summarize
results = result.groupby(['date','title'])['temperature'].mean().reset_index()

# merge
results = results.merge(result, on=['title','date'], how='right')




In [39]:
results.drop(columns=['temperature_y','index_right'], inplace=True)

In [40]:
#rename
results.rename(columns={'temperature_x':'temperature'}, inplace=True)

# see the head
results.head()

,date,title,temperature,soil_quality,battles,explosions,violence,fatalities_battles,fatalities_explosions,fatalities_violence,...,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,elevation,soil,price,GOSIF_GPP,chirps_rainfall,geometry
0,2022-06-01,//Kharas,296.818182,142.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15,0.45,0.45,0.45,988.0,143.0,626.811667,666.215772,3.484402,"POLYGON ((15.79811 -25.10764, 15.88710 -25.605..."
1,2022-06-01,//Kharas,296.818182,142.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15,0.45,0.45,0.45,988.0,143.0,626.811667,666.215772,3.484402,"POLYGON ((15.79811 -25.10764, 15.88710 -25.605..."
2,2022-06-01,//Kharas,296.818182,142.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15,0.45,0.45,0.45,988.0,143.0,626.811667,666.215772,3.484402,"POLYGON ((15.79811 -25.10764, 15.88710 -25.605..."
3,2022-06-01,//Kharas,296.818182,142.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15,0.45,0.45,0.45,988.0,143.0,626.811667,666.215772,3.484402,"POLYGON ((15.79811 -25.10764, 15.88710 -25.605..."
4,2022-06-01,//Kharas,296.818182,142.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.15,0.45,0.45,0.45,988.0,143.0,626.811667,666.215772,3.484402,"POLYGON ((15.79811 -25.10764, 15.88710 -25.605..."


In [41]:
#drop duplicates
results.drop_duplicates(inplace=True)

In [42]:
# describe
results.describe()

,date,temperature,soil_quality,battles,explosions,violence,fatalities_battles,fatalities_explosions,fatalities_violence,estimated_population,...,phase3_worse_percentage_manual,phase3_plus_phase4,phase2_worse_percentage_manual,phase2_plus_phase3,phase2_plus_phase3_plus_phase4,elevation,soil,price,GOSIF_GPP,chirps_rainfall
count,5434,5434.000000,4789.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5.434000e+03,...,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000,5434.000000
mean,2020-01-25 12:31:16.186971136,293.514518,136.422009,123.737026,61.964851,71.780272,571.798675,533.339161,564.660655,1.088735e+06,...,0.444107,0.445024,0.933432,0.847196,0.937344,1100.559257,101.960802,1235.328927,20019.716139,109.380924
min,2016-10-01 00:00:00,267.636364,90.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000e+00,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,80.000000,0.000000,0.000000
25%,2018-07-01 00:00:00,291.666667,129.000000,1.000000,0.000000,4.000000,0.000000,0.000000,3.000000,2.141520e+05,...,0.150000,0.150000,0.500000,0.490000,0.500000,401.000000,0.000000,486.666667,2980.333333,6.246937
50%,2020-04-01 00:00:00,295.000000,138.000000,20.000000,2.000000,16.000000,71.000000,2.000000,71.000000,5.067940e+05,...,0.350000,0.350000,0.750000,0.650000,0.750000,950.000000,0.000000,735.472000,11039.571429,37.973124
75%,2021-09-01 00:00:00,300.222222,145.000000,90.000000,55.000000,74.000000,417.500000,268.000000,411.000000,1.110213e+06,...,0.550000,0.550000,1.200000,1.100000,1.200000,1470.000000,149.000000,1423.750000,28615.238899,152.142559
max,2022-12-01 00:00:00,305.800000,161.000000,1496.000000,784.000000,2330.000000,10802.000000,10802.000000,10802.000000,1.402262e+07,...,3.800000,3.200000,6.000000,5.750000,6.000000,7990.000000,786.000000,43532.780000,199047.142857,1861.817209
std,NaN,9.422563,10.586361,267.696225,146.587769,172.456745,1581.614173,1587.374642,1579.701871,1.557112e+06,...,0.413271,0.407018,0.691811,0.591417,0.685759,1009.628270,133.533946,1847.560254,22857.861130,164.195725


In [44]:
results.columns

Index(['date', 'title', 'temperature', 'soil_quality', 'battles', 'explosions',
       'violence', 'fatalities_battles', 'fatalities_explosions',
       'fatalities_violence', 'id', 'estimated_population', 'anl_id',
       'overall_phase', 'country', 'year', 'condition', 'phase1_population',
       'phase1_percent', 'phase2_population', 'phase2_percent',
       'phase3_population', 'phase3_percent', 'phase4_population',
       'phase4_percent', 'phase5_population', 'phase5_percent',
       'phase3_worse_percentage_manual', 'phase3_plus_phase4',
       'phase2_worse_percentage_manual', 'phase2_plus_phase3',
       'phase2_plus_phase3_plus_phase4', 'elevation', 'soil', 'price',
       'GOSIF_GPP', 'chirps_rainfall', 'geometry'],
      dtype='object')

In [46]:
# convert dataframe to geodataframe
results = gpd.GeoDataFrame(results, geometry='geometry')

# save as geojson
results.to_file(r'C:\Users\WeilunShi\OneDrive - CGIAR\Desktop\Food Crisis and Insecurity\\2.merge_data\output6.geojson', driver='GeoJSON')